In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
from os import path
import shutil
import tabula
import glob
import pikepdf
from selenium.webdriver.support.select import Select
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import io
import re


In [2]:
mylist = '''CONTACT
TERMINAL 
PARALLEL CONNECTOR
COPPER NGR
COPPER HVTG
COPPER CONECTIONS
CASTING
DRILL
END MILL
SAW
HSS SLITTING SAW
HSS
PRESS BOARD
ARALDITE
Cotton Tape Electical purpose
INSULAT
VARNISH
'''
mylist = mylist.replace('\n', ',')
mylist = mylist.split(',')


In [53]:
def latestDownload(src):
    """ sorting the file according to the time its being made and returing the 
    latest downloaded file in the directory"""
    path = src
    os.chdir(path)
    files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
    newest = files[-1]

    return newest


### Script to move file from destination to source gem folder

In [50]:
def move_file(src, dst,keyword="GEM"):
    """find the latest files which start with the name GEM in the source downoad 
    folder and move it to the destination folder where the data extraation and downloaidng
    the link happens . Please the source , destination , key word of the
     file starting to take the action
    """
    files = [i for i in os.listdir(src) if i.startswith(
        keyword) and path.isfile(path.join(src, i))]
    for f in files:
        shutil.move(os.path.join(src, f), os.path.join(dst, f))
    return files


### Script to download the respective file fron GEM website using selenium

In [5]:
def get_driver():
    """ function to get the selenium edge driver"""
    ser = Service(executable_path='C:\\edgedriver_win64\\msedgedriver.exe')
    driver = webdriver.Edge(service=ser)
    driver.maximize_window()
    return driver


In [56]:
def itemwise_download(url,items,src):
    """ """
    driver=get_driver()
    driver.get(url)
    driver.find_element(By.XPATH,'//*[@id="search_concept"]').click()
    time.sleep(1)                    
    driver.find_element(By.XPATH,'//*[@id="exTab2"]/div[1]/div[2]/form/div/div/ul/li[2]/a').click()
    time.sleep(1)
    
    for item in items:
        Searchbox = driver.find_element(By.XPATH,('//*[@id="search_by"]'))
        Searchbox.clear()    
        Searchbox.send_keys(item)
        driver.find_element(By.XPATH,'//*[@id="exTab2"]/div[1]/div[2]/form/div/span/button/span').click()
    
        linklist=[]
        for j in range(1,10):
            try:
                # Extracting the text from the link element
                link=driver.find_element(By.XPATH,f'//*[@id="pagi_content"]/div[{j}]/div[1]/p[1]/a').text
                linklist.append(link)
            
            except Exception as e  :
                print ( "error",str(e))
                pass
        
        if (len(linklist))>0:
            
            for k in range(1,len(linklist)+1):

                # Extracting the text from the link element
                link=driver.find_element(By.XPATH,f'//*[@id="pagi_content"]/div[{k}]/div[1]/p[1]/a').text
                link=link.replace("/","")
                link=f'{link}.pdf'
                # click on the link and download the file to destination.
                driver.find_element(By.XPATH,f'//*[@id="pagi_content"]/div[{k}]/div[1]/p[1]/a').click()

                time.sleep(2) 
                #identifying the latest downloaded file in source directory
                file=latestDownload(src)
                           
                if file.startswith("GeM") and path.isfile(path.join(src, file)):
                    os.rename(file,link)
                    move_file(src,dst)
                                

        else:
                continue

        time.sleep(2)

    
    return link
    
    

### script to extract text from pdf

In [7]:
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)

            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()


def extract_text(pdf_path):
    tot = {}
    for i, page in enumerate(extract_text_by_page(pdf_path)):
        tot.update({i+1: page})
    return tot


#### Function to find list of PDF file in the folder

In [57]:
def pdf_list(dst,extension='pdf'):
    """ returns the list of files with any extension from a folder"""
    pdf_file_list = glob.glob(f'{dst}/*.{extension}')
    return pdf_file_list


In [9]:
def dataExtraction(pattern, pdftext):
    """using regex extarcting data from the pdf file """

    pattern_data = re.findall(pattern, pdftext)
    try:
        pattern_data = pattern_data[0]
    except Exception as e:
        pattern_data = "NA"
        print("error", str(e))
        pass
    return pattern_data


### data extract to a CSV 

In [16]:
def data_to_csv(pdf_file_list):

    for a in pdf_file_list:

        tender_no = a.split('\\')[1].replace('.pdf', '')
        df = tabula.read_pdf(a, pages="1", stream=True)
        pdftext = str(extract_text(a))
        
        date_pattern = 'Dated: (\d{2}-\d{2}-\d{4})'
        tender_date = dataExtraction(date_pattern, pdftext)

        place_pattern = 'AddressQuantityDelivery Days1(.*?)Buyer'
        place = dataExtraction(place_pattern, pdftext)

        Emd_pattern = 'DocumentEMD DetailRequired([a-zA-Z]*)'
        Emd = dataExtraction(Emd_pattern, pdftext)

        ePBG_pattern = 'ePBG DetailRequired([a-zA-Z]*)'
        ePBG = dataExtraction(ePBG_pattern, pdftext)

        place1_pattern = 'Delivery Days1\**([\w][\D]*)'
        place1 = dataExtraction(place1_pattern, pdftext)

        try:
            mseindex = df[0][df[0]['Unnamed: 0'].str.contains(
                'MSE', regex=False, case=False, na=False)].index.tolist()
            msevalue = df[0]['Bid Details'][mseindex[0]+1]
        except Exception as e:
            msevalue = 'No'
            print("error", str(e))
            pass
        new_df = [[
            tender_no, tender_date, place, place1, ePBG, Emd,
            df[0]['Bid Details'][0],  # Bid End
            df[0]['Bid Details'][1],  # Bid Open
            df[0]['Bid Details'][4],  # Ministry Name
            df[0]['Bid Details'][5],  # Department Name
            df[0]['Bid Details'][6],  # Oranization name
            df[0]['Bid Details'][7],  # Office Name
            df[0]['Bid Details'][8],  # Total Qunatity
            df[0]['Bid Details'][9],  # Item Category
            msevalue

        ]]
        columns = ['tender_no', 'Date', 'place_of_supply', 'place1',
                   'ePBG', 'Emd', 'Bid End Date/Time',
                   'Bid Opening Date/Time',
                   'Ministry Name', 'Department Name',
                   'Organisation Name', 'office Name',
                   'Total Quantity', 'Item Category', 'MSE Exemption']

        new_df = pd.DataFrame(new_df, columns=columns)

        if os.path.isfile("D:\\Gem1\\gem1.csv"):
            new_df.to_csv("D:\\Gem1\\gem1.csv", index=False,
                          header=False, mode='a')
        else:
            new_df.to_csv("D:\\Gem1\\gem1.csv", index=False)
    return new_df


In [45]:
def tenderNum(file):
    
    tender_no = file.split('\\')[1].replace('.pdf', '')
    return tender_no

In [39]:
def tenderFolder(file,src):           
    tender_no = tenderNum(file)
    directory = tender_no
    folder_location = os.path.join(src, directory)
    if not os.path.exists(folder_location):
        os.mkdir(folder_location)
    yield folder_location


### download link file from pdf list

In [48]:
def link_download(pdf_file_list):
    urls = []        
    for file in pdf_file_list:
        tender_no = tenderNum(file)
        folder_location=next(tenderFolder(file))
        pdf_file = pikepdf.Pdf.open(file)
        for page in pdf_file.pages:
            for annots in page.get("/Annots"):
                try:
                    uri = annots.get("/A").get("/URI")
                    urls.append(uri)
                except Exception as e:
                    print("error", str(e))
                    pass
                if uri is not None:
                    #print("[+] URL founs",uri)
                    # print(str(uri))
                    # do not download the file with terms and conditions
                    if str(uri).endswith("termsCondition"):
                        pass
                    else:
                        # Download the CSV, PDF, WOrd or excel file
                        if str(uri).endswith(".pdf") or str(uri).endswith(".csv") or str(uri).endswith(".docx") or str(uri).endswith(".xlsx"):
                            filename = os.path.join(
                                folder_location, str(uri).split("/")[-1])
                            r = requests.get(str(uri), stream=True)
                            print("Saving to", filename)
                            with open(filename, 'wb') as f:
                                f.write(r.content)

                        else:
                            try:
                                # Download the table directly from website
                                html = requests.get(str(uri))
                                soup = BeautifulSoup(html.text, 'lxml')
                                tables = soup.find_all('table')
                                df = pd.read_html(str(tables[0]))[0]
                                folder = f'D:\\Gem\\{tender_no}\\'
                                df.to_csv(f"{folder}\\{tender_no}.csv")
                            except Exception as e:
                                print("error", str(e))
                                pass

        pdf_file.close()
    return urls


### download only link file from pdf list

In [43]:
def linkOnlyDownloader(pdf_file_list):
    urls = []
    for file in pdf_file_list:
        tender_no = tenderNum(file)
        #folder_location=next(tenderFolder(file))
        pdf_file = pikepdf.Pdf.open(file)
        
        for page in pdf_file.pages:
            for annots in page.get("/Annots"):
                try:
                    uri = annots.get("/A").get("/URI")
                except Exception as e:
                    print("error", str(e))
                    pass

                if uri is not None:
                    #print("[+] URL founs",uri)
                    # print(str(uri))
                    # do not download the file with terms and conditions
                    new_df = [[tender_no, str(uri)]]
                    columns = ['tender_no', 'link']
                    new_df = pd.DataFrame(new_df, columns=columns)
                    if os.path.isfile("D:\\Gem1\\link.csv"):
                        new_df.to_csv("D:\\Gem1\\link.csv",
                                      index=False, header=False, mode='a')
                    else:
                        new_df.to_csv("D:\\Gem1\\link.csv", index=False)

        pdf_file.close()
    return urls


### move the tenderfile to it own folders

In [23]:
def move_pdf_file(pdflist):
    for file in pdflist:
        source = file
        tender_no = file.split('\\')[1].replace('.pdf', '')

        # Destination path
        destination = f"D:\\Gem2\\{tender_no}"

        # Move the content of
        # source to destination
        dest = shutil.move(source, destination)

        # print(dest) prints the
        # Destination of moved directory
    return (print("files moved"))


### final script to run

In [ ]:
src = "C:/Users/Admin/Downloads"
dst = "D:/Gem1/"
items = mylist
url = 'https://bidplus.gem.gov.in/custom-item'
itemwise_download(url, items, src)
pdf_file_list = pdf_list(dst)
data_to_csv(pdf_file_list)
link_download(pdf_file_list)
move_pdf_file(pdf_file_list)


## Testing

In [22]:
dst = "D:/Gem1/"
pdf_file_list = pdf_list(dst)
move_pdf_file(pdf_file_list)


files moved


In [24]:
src = "C:/Users/Admin/Downloads"
dst = "D:/Gem1/"
items = mylist
pdf_file_list = pdf_list(dst)
